# 1. Movie Recommender System – Data Preparation
This section prepares the dataset by importing, merging, cleaning, and selecting relevant features.


In [ ]:
# Importing required libreries and Datasets

import numpy as np 
import pandas as pd

#  Load Datasets
We use TMDB datasets containing movie metadata and credits.


In [ ]:
# read datasets
movies_raw = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Merge Datasets
Merge movies and credits on the 'title' column to unify metadata.


In [ ]:
movies = movies_raw.merge(credits, on = 'title')

In [ ]:
movies.head(1)

# Select Relevant Features
Keep only the columns required for building the recommender system.


In [ ]:
# Selecting relevant features
movies = movies[['movie_id','title','genres','keywords','overview','cast','crew']]

# Handle Missing Values and duplicate values
Drop rows with missing values to ensure clean data and remove duplicate values 


In [ ]:
#remove missing values
movies.dropna(inplace = True)

In [ ]:
#check for missing values
movies.isnull().sum()

In [ ]:
#removie duplicates
movies.drop_duplicates()

In [ ]:
#check for duplicates
movies.duplicated().sum()

# 2.Text preprocessing
doing this we will convert JSON like strings into list of names

In [ ]:
import ast 

In [ ]:
#we will create a helper function to convert the JSON like strings into list of names using ast
def convert(obj):
    return [i['name'] for i in ast.literal_eval(obj)]

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
 #Function for cast as there is a lot of names that will lead us to wrong recommendation
def convert_cast(obj):
    return [i['name'] for i in ast.literal_eval(obj)[ : 4]]
        

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
#function to fetch the name of director ofthe movie
def fetch_director(obj):
      return [i['name'] for i in ast.literal_eval(obj) if i['job'] == 'Director']
            

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
#now we must convert the overview into list from string as we have to concatnate it with other 

movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
#now we will remove spaces as we have treat every name and words which represents one person or thing but have more than two entities , we will make one entity so it wont cause mislead to recomendation
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
# checking the converted data
movies.head(1)

In [ ]:
#now we merge all 4 coluumns in one named tags so the complexicity will be reduced and similarity calculated easily
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['overview']

In [ ]:
movies['tags']

In [ ]:
movies = movies[['movie_id','title','tags']]

In [ ]:
movies

# Text Normalization

In [ ]:
# Converting tags list into string and lowercase it to make vectorizable
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x).lower())

In [ ]:
movies.head(1)

# 3.Vectorization
 Here We Will Convert text into numerical vectors using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)

tfidf = vectorizer.fit_transform(movies["tags"])


In [ ]:
#checking vectors
tfidf

In [ ]:
vectorizer.get_feature_names_out()

# Stemming 

In [ ]:
import nltk

In [ ]:
#there is our list but here we will see some similar words like 'action' and 'actions' basically they are same but trated as different 
#which will give us a lengthy data 
#so to make it a base word what actually it means we will use stemming from nltk library

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
#function for stemming
def stemming(text):
    return " ".join(ps.stem(i) for i in text.split())
        

In [ ]:
movies['tags'] = movies['tags'].apply(stemming)

# Dumping Files For UI On Streamlit

In [ ]:
import pickle

In [ ]:
pickle.dump(movies , open('movies.pkl','wb'))

In [ ]:
pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [ ]:
pickle.dump(vectorizer,open('vectorizer.pkl','wb'))

In [ ]:
movies